In [ ]:
#This did not include the original latitude and longitude
#Altered code in next cell to include - see below

import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

# Declare API key and client info
FOURSQUARE_API_KEY = os.getenv("FOURSQUARE_API_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

# URL for places query
base_url = 'https://api.foursquare.com/v3/places/search'

# Load dataframe from CSV
montreal_bike_data = pd.read_csv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/montreal_df.csv')

# Initialize empty list to store data for DataFrame
data_for_df = []

# Parameters for API request
params = {
    'query': 'restaurant',
    'radius': 1000,
    'limit': 10,
    'sort': 'rating'
}

headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

# Loop through dataframe to make API requests
for index, row in montreal_bike_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    ll_joined = f"{latitude},{longitude}"
    params['ll'] = ll_joined

    try:
        response = requests.get(base_url, params=params, headers=headers, timeout=30)
        response.raise_for_status()

        result_data = response.json()
        
        # Extract relevant information from the result_data
        for result in result_data.get('results', []):
            venue_name = result.get('name')
            address = result['location'].get('formatted_address')
            latitude = result['geocodes']['main']['latitude']
            longitude = result['geocodes']['main']['longitude']
            categories = [cat['name'] for cat in result['categories']]
            rating = result.get('rating', None)
            total_ratings = result['stats']['total_ratings'] if 'stats' in result else None
        
        #get further details to calculate a 
            
          # Append extracted data to data_for_df as a dictionary
            data_for_df.append({
                'Name': venue_name,
                'Address': address,
                'Latitude': latitude,
                'Longitude': longitude,
                'Categories': categories,
                'Rating': rating,
                'TotalRatings': total_ratings
            })
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed for latitude {latitude}, longitude {longitude}. Error: {e}")

# Convert the list of dictionaries into a DataFrame
foursquare_df = pd.DataFrame(data_for_df)

# Display the DataFrame
print(foursquare_df.head())
foursquare_df.to_csv('foursquare_data.csv', index=False)

In [1]:
#Found that the original latitude and longitude were not stored to be referenced in merge.
import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

# Declare API key and client info
FOURSQUARE_API_KEY = os.getenv("FOURSQUARE_API_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

# URL for places query
base_url = 'https://api.foursquare.com/v3/places/search'

# Load dataframe from CSV
montreal_bike_data = pd.read_csv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/montreal_df.csv')


# Initialize empty list to store data for DataFrame
data_for_df = []

# Parameters for API request
params = {
    'query': 'restaurant',
    'radius': 1000,
    'limit': 10,
    'sort': 'rating'
}

headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

# Loop through dataframe to make API requests
for index, row in montreal_bike_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    ll_joined = f"{latitude},{longitude}"
    params['ll'] = ll_joined

    try:
        response = requests.get(base_url, params=params, headers=headers, timeout=30)
        response.raise_for_status()

        result_data = response.json()
        
        # Extract relevant information from the result_data
        for result in result_data.get('results', []):
            venue_name = result.get('name')
            address = result['location'].get('formatted_address')
            venue_latitude = result['geocodes']['main']['latitude']  # Latitude of the venue
            venue_longitude = result['geocodes']['main']['longitude']  # Longitude of the venue
            categories = [cat['name'] for cat in result['categories']]
            rating = result.get('rating', None)
            total_ratings = result['stats']['total_ratings'] if 'stats' in result else None
        
            # Append extracted data to data_for_df as a dictionary
            data_for_df.append({
                'Original_Latitude': latitude,  # Original latitude from montreal_bike_data
                'Original_Longitude': longitude,  # Original longitude from montreal_bike_data
                'Venue_Name': venue_name,
                'Address': address,
                'Latitude': venue_latitude,  # Latitude of the venue from Foursquare
                'Longitude': venue_longitude,  # Longitude of the venue from Foursquare
                'Categories': categories,
                'Rating': rating,
                'TotalRatings': total_ratings
            })
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed for latitude {latitude}, longitude {longitude}. Error: {e}")

# Convert the list of dictionaries into a DataFrame
foursquare_df = pd.DataFrame(data_for_df)

# Display the DataFrame
print(foursquare_df.head())

# Save DataFrame to CSV
foursquare_df.to_csv('foursquare_data.csv', index=False)

KeyboardInterrupt: 

In [13]:
foursquare_df.columns

Index(['Original_Latitude', 'Original_Longitude', 'Venue_Name', 'Address',
       'Latitude', 'Longitude', 'Categories', 'Rating', 'TotalRatings'],
      dtype='object')

In [15]:
null_rating_count = foursquare_df['TotalRatings'].notna().sum()

In [16]:
null_rating_count


0

In [17]:
foursquare_df

,Original_Latitude,Original_Longitude,Venue_Name,Address,Latitude,Longitude,Categories,Rating,TotalRatings
0,45.516926,-73.564257,Cadet,1431 Saint-Laurent Blvd (entre De Maisonneuve ...,45.510388,-73.564365,"[Restaurant, Wine Bar]",None,None
1,45.516926,-73.564257,L'Express,"3927 Saint-Denis Rue (entre Duluth & Roy), Mon...",45.520186,-73.574269,"[Bistro, French Restaurant]",None,None
2,45.516926,-73.564257,Le Club Espresso Bar,"3801 Saint-Denis Rue, Montréal QC H2W 2M4",45.519481,-73.572648,[Café],None,None
3,45.516926,-73.564257,Bouillon Bilk,1595 Saint-Laurent Blvd (entre De Maisonneuve ...,45.511038,-73.565936,[Restaurant],None,None
4,45.516926,-73.564257,Pourquoi Pas Espresso Bar,1447 Atateken Rue (entre De Maisonneuve & Ste-...,45.517964,-73.557776,"[Café, Coffee Shop, Restaurant]",None,None
...,...,...,...,...,...,...,...,...,...
8002,45.559887,-73.698220,Allo Pizza,"105 Cartier Blvd O, Laval QC H7N 2H6",45.560727,-73.690496,[Pizzeria],None,None
8003,45.559887,-73.698220,Restaurant le Cartier,"319 Cartier Blvd O (Ampère), Laval QC H7N 2J4",45.555011,-73.702614,[Steakhouse],None,None
8004,45.559887,-73.698220,Café Bistro Il Momento,"180 Blvd de la Concorde Ouest, Laval QC H7N 1H9",45.565933,-73.699552,"[Bistro, Café, Coffee Shop]",None,None
8005,45.559887,-73.698220,Rôtisseries Benny & Au Coq,"54 Cartier Blvd O, Laval QC",45.559354,-73.689475,[Fried Chicken Joint],None,None
